In [60]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn  
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, make_scorer
from keras import layers, metrics, optimizers, models, losses, utils
from keras.preprocessing import image_dataset_from_directory
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np  
from PIL import Image
import glob

In [61]:
sklearn.set_config(transform_output="pandas")

In [68]:

dir = r"C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Data\LEGO brick images v1"

In [100]:
df= image_dataset_from_directory(
    directory = dir,
    labels='inferred', 
    label_mode='categorical',
    color_mode="grayscale",
    image_size=(200, 200),
    batch_size = 128,
    shuffle=True,
    seed=42,
    interpolation="bilinear",
    validation_split=0.2,
    subset='training'
)

Found 6379 files belonging to 16 classes.
Using 5104 files for training.


In [101]:
df_val= image_dataset_from_directory(
    directory = dir,
    labels='inferred', 
    label_mode='categorical',
    color_mode="grayscale",
    image_size=(200, 200),
    batch_size = 128,
    shuffle=True,
    seed=42,
    interpolation="bilinear",
    validation_split=0.2,
    subset='validation'
)

Found 6379 files belonging to 16 classes.
Using 1275 files for validation.


In [102]:
print(df_val.class_names)

['11214 Bush 3M friction with Cross axle', '18651 Cross Axle 2M with Snap friction', '2357 Brick corner 1x2x2', '3003 Brick 2x2', '3004 Brick 1x2', '3005 Brick 1x1', '3022 Plate 2x2', '3023 Plate 1x2', '3024 Plate 1x1', '3040 Roof Tile 1x2x45deg', '3069 Flat Tile 1x2', '32123 half Bush', '3673 Peg 2M', '3713 Bush for Cross Axle', '3794 Plate 1X2 with 1 Knob', '6632 Technic Lever 3M']


In [103]:
print(df.class_names)

['11214 Bush 3M friction with Cross axle', '18651 Cross Axle 2M with Snap friction', '2357 Brick corner 1x2x2', '3003 Brick 2x2', '3004 Brick 1x2', '3005 Brick 1x1', '3022 Plate 2x2', '3023 Plate 1x2', '3024 Plate 1x1', '3040 Roof Tile 1x2x45deg', '3069 Flat Tile 1x2', '32123 half Bush', '3673 Peg 2M', '3713 Bush for Cross Axle', '3794 Plate 1X2 with 1 Knob', '6632 Technic Lever 3M']


In [104]:
labels = df.class_names

In [105]:
print(labels)

['11214 Bush 3M friction with Cross axle', '18651 Cross Axle 2M with Snap friction', '2357 Brick corner 1x2x2', '3003 Brick 2x2', '3004 Brick 1x2', '3005 Brick 1x1', '3022 Plate 2x2', '3023 Plate 1x2', '3024 Plate 1x1', '3040 Roof Tile 1x2x45deg', '3069 Flat Tile 1x2', '32123 half Bush', '3673 Peg 2M', '3713 Bush for Cross Axle', '3794 Plate 1X2 with 1 Knob', '6632 Technic Lever 3M']


In [106]:
print(type(df))

<class 'tensorflow.python.data.ops.prefetch_op._PrefetchDataset'>


In [133]:
model = Sequential(
       layers=(
           
           Conv2D(filters=16, kernel_size=(3,3), activation='relu', input_shape=(200, 200, 1)),
           MaxPooling2D(2,2),
           
           Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
           MaxPooling2D(2,2),
           
           Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
           MaxPooling2D(2,2),
           
           Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
           MaxPooling2D(2,2),
           
           Flatten(),
           Dense(64,activation='relu'),
           Dense(32, activation='relu'),
           Dense(16, activation='softmax')
           
           )
       )



c:\Users\BrunoLuciano\miniconda3\envs\UFS12DL\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [134]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
                metrics=['accuracy'])


In [135]:
epoche = 10

In [136]:
model.fit(df,epochs=epoche,validation_data=df_val,batch_size=32)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.1092 - loss: 4.5792 - val_accuracy: 0.1294 - val_loss: 2.4546
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 63s 2s/step - accuracy: 0.1568 - loss: 2.3071 - val_accuracy: 0.2596 - val_loss: 1.9926
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 65s 2s/step - accuracy: 0.3048 - loss: 1.8853 - val_accuracy: 0.3600 - val_loss: 1.7631
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.4450 - loss: 1.5779 - val_accuracy: 0.5239 - val_loss: 1.4609
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.5441 - loss: 1.3380 - val_accuracy: 0.5122 - val_loss: 1.4237
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.5953 - loss: 1.2132 - val_accuracy: 0.5686 - val_loss: 1.2851
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 71s 2s/step - accuracy: 0.6267 - loss: 1.0676 - val_accuracy: 0.5647 - val_loss: 1.2497
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 67s 2s/step - accuracy: 0.6412 - loss: 0.9951 - val_accuracy: 0.6188 - val_loss:

In [137]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_23 (Conv2D)              │ (None, 198, 198, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_23 (MaxPooling2D) │ (None, 99, 99, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_24 (Conv2D)              │ (None, 97, 97, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_24 (MaxPooling2D) │ (None, 48, 48, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 46, 46, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_25 (MaxPooling2D) │ (None, 23, 23, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 21, 21, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_26 (MaxPooling2D) │ (None, 10, 10, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 12800)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 32)             │       409,632 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 16)             │           528 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,521,938 (5.81 MB)

 Trainable params: 507,312 (1.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,014,626 (3.87 MB)

In [138]:
model.save(filepath=r'C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Testing')

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `tf.saved_model.save()` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=C:\Users\BrunoLuciano\OneDrive - ITS Angelo Rizzoli\Documents\Deep_learning_project\Testing.